In [1]:
import tensorflow as tf
from tensorflow.layers import dense, flatten, conv2d, max_pooling2d
from utils import reshape_train_var
import gym
import numpy as np
from rewards import discount_rewards
from memory import MemoryBuffer, MTMemoryBuffer
from policies import VPGTrainer
from threading import Thread
import time
import math
from multiprocessing import Process
from env import *

In [12]:
class PPOTrainer():
    def __init__(self, in_op, out_op, value_out_op, act_type='discrete', sess=None, clip_val=0.2, ppo_iters=30,
                 actor_lr=3e-4, value_lr=1e-3):
        """
        Create a wrapper for RL networks for easy training.
        Args:
            in_op (tf.Placeholder): Observation input to architecture
            out_op (tf.Variable): Action output of architecture
            act_type (string): 'discrete' for a discrete actions space or 'continuous'
                               for a continuous actions space
            sess (tf.Session): A session if you would like to use a custom session,
                               if left none it will be automatically created
        """

        if not sess:
            self.renew_sess()
        
        self.in_op = in_op
        self.out_op = out_op
        self.value_out_op = value_out_op
        self._prev_weights = None
        self.clip_val = clip_val
        self.ppo_iters = ppo_iters
        self.actor_lr=3e-4 
        self.value_lr=1e-3
        
        if act_type in ('discrete', 'd'):
            self.train = self._create_discrete_trainer()
            self.act_type = 'discrete'
        elif act_type in ('continuous', 'c'):
            self.train = self._create_continuous_trainer()
            self.act_type = 'continuous'
        else:
            raise TypeError('act_type must be \'discrete\' or \'continuous\'')
        
    def renew_sess(self):
        """
        Starts a new internal Tensorflow session
        """
        self.sess = tf.Session()
        
    def end_sess(self):
        """
        Ends the internal Tensorflow session if it exists
        """
        if self.sess:
            self.sess.close()
        
    def _create_discrete_trainer(self, optimizer=tf.train.AdamOptimizer()):
        """
        Creates a function for vanilla policy training with a discrete action space
        """
        # First passthrough
        
        self.act_holders = tf.placeholder(tf.int64, shape=[None])
        self.reward_holders = tf.placeholder(tf.float64, shape=[None])
        
        self.act_masks = tf.one_hot(self.act_holders, self.out_op.shape[1].value, dtype=tf.float64)
        self.resp_acts = tf.reduce_sum(self.act_masks *  self.out_op, axis=1)
        
        self.advantages = self.value_out_op - self.reward_holders
        
        # Second passthrough
        
        self.advatange_holders = tf.placeholder(dtype=tf.float64, shape=self.advantages.shape)
        self.old_prob_holders = tf.placeholder(dtype=tf.float64, shape=self.resp_acts.shape)
 
        self.policy_ratio = tf.reduce_mean(self.resp_acts / self.old_prob_holders)
        self.clipped_ratio = tf.clip_by_value(self.policy_ratio, 1 - self.clip_val, 1 + self.clip_val)

        self.min_loss = tf.minimum(self.policy_ratio * self.advatange_holders, self.clipped_ratio * self.advatange_holders)
        self.actor_loss = tf.reduce_mean(self.min_loss)

        self.actor_optimizer = tf.train.AdamOptimizer(learning_rate=self.actor_lr)
        self.actor_update = self.actor_optimizer.minimize(self.actor_loss)

        # Value update
        
        self.value_optimizer = tf.train.AdamOptimizer(learning_rate=self.value_lr)
        self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - self.value_out_op))
        self.value_update = self.value_optimizer.minimize(self.value_loss)
        
        def update_func(train_data):
            self.old_probs, self.old_advantages = self.sess.run([self.resp_acts, self.advantages], 
                                    feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                               self.act_holders: train_data[:, 1],
                                               self.reward_holders: train_data[:, 2]})
            
            for i in range(self.ppo_iters):
                self.sess.run([self.actor_update], 
                               feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                    self.act_holders: train_data[:, 1],
                                    self.old_prob_holders: self.old_probs,
                                    self.advatange_holders: self.old_advantages})
            
                self.sess.run([self.value_update], 
                               feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                          self.reward_holders: train_data[:, 2]})

        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _create_continuous_trainer(self):
        """
        Creates a function for vanilla policy training with a continuous action space
        """
        # First passthrough
        
        self.reward_holders = tf.placeholder(tf.float64, shape=[None])
        
        self.advantages = self.value_out_op - self.reward_holders
        
        # Second passthrough
        
        self.advatange_holders = tf.placeholder(dtype=tf.float64, shape=self.advantages.shape)
        self.old_prob_holders = tf.placeholder(dtype=tf.float64, shape=self.out_op.shape)

        self.policy_ratio = tf.reduce_mean(self.out_op / self.old_prob_holders, axis=1)
        self.clipped_ratio = tf.clip_by_value(self.policy_ratio, 1 - self.clip_val, 1 + self.clip_val)

        self.min_loss = tf.minimum(self.policy_ratio * self.advatange_holders, self.clipped_ratio * self.advatange_holders)
        self.actor_loss = tf.reduce_mean(self.min_loss)

        self.actor_optimizer = tf.train.AdamOptimizer(learning_rate=self.actor_lr)
        self.actor_update = self.actor_optimizer.minimize(self.actor_loss)

        # Value update
        
        self.value_optimizer = tf.train.AdamOptimizer(learning_rate=self.value_lr)
        self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - self.value_out_op))
        self.value_update = self.value_optimizer.minimize(self.value_loss)
        
        def update_func(train_data):
            self.old_probs, self.old_advantages = self.sess.run([self.out_op, self.advantages], 
                                    feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                               self.reward_holders: train_data[:, 2]})
            
            for i in range(self.ppo_iters):
                self.sess.run([self.actor_update], 
                               feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                    self.old_prob_holders: self.old_probs,
                                    self.advatange_holders: self.old_advantages})
            
            self.sess.run([self.value_update], 
                           feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                      self.reward_holders: train_data[:, 2]})

        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _gen_discrete_act(self, obs):
        act_probs = self.sess.run(self.out_op, feed_dict={self.in_op: obs})
        act = np.random.choice(list(range(len(act_probs)+1)), p=act_probs[0])
        
        return act
    
    def _gen_continuous_act(self, obs):
        act_vect = self.sess.run(self.out_op, feed_dict={self.in_op: obs})[0]
        
        # TODO: Add gaussian noise t128o action vector
        act_vect = [a + np.random.normal(0., 0.1) for a in act_vect]
        
        return np.array(act_vect)
        
    def gen_act(self, obs):
        if self.act_type == 'discrete':
            return self._gen_discrete_act(obs)
        else:
            return self._gen_continuous_act(obs)
        
    def train(self, obs, rewards, acts):
        raise RuntimeError('The train method was not properly created')

In [13]:
# Cart pole network

env = make_cart_pole()

obs = tf.placeholder(tf.float64, shape=[None, 4])
dense1 = dense(obs, 10, activation=tf.tanh)
dense2 = dense(dense1, 10, activation=tf.tanh)
act_probs = dense(dense2, 2)
softmax_probs = tf.nn.softmax(act_probs)

v_dense1 = dense(obs, 10, activation=tf.tanh)
v_dense2 = dense(v_dense1, 10, activation=tf.tanh)
value = dense(v_dense2, 1)

network = PPOTrainer(obs, softmax_probs, value, act_type='discrete', ppo_iters=80)

In [14]:
# Car race network

# obs = tf.placeholder(tf.float64, shape=np.concatenate([[None], env.observation_space.shape]))
# conv1 = conv2d(obs, 16, [5, 5], padding='valid', activation=tf.tanh)
# maxpool1 = max_pooling2d(conv1, 4, 4)
# conv2 = conv2d(maxpool1, 32, [3, 3], padding='valid', activation=tf.tanh)
# maxpool2 = max_pooling2d(conv2, 2, 2)
# flatten1 = flatten(maxpool2)
# act_vects = dense(flatten1, env.action_space.shape[0])

# network = VanillaPolicyTrainer(obs, act_vects, act_type='continuous')

In [15]:
# Lunar lander discrete network

# obs = tf.placeholder(tf.float64, shape=[None, env.observation_space.shape[0]])
# dense1 = dense(obs, 64, activation=tf.nn.tanh)
# dense2 = dense(dense1, 64, activation=tf.nn.tanh)
# act_probs = dense(dense2, 2)
# softmax_probs = tf.nn.softmax(act_probs)

# network = VanillaPolicyTrainer(obs, softmax_probs, act_type='d')

In [16]:
# # Lunar lander continuous network

# env = make_lunar_lander_c()

# obs = tf.placeholder(tf.float64, shape=[None, env.observation_space.shape[0]])
# dense1 = dense(obs, 32, activation=tf.nn.tanh)
# dense2 = dense(dense1, 32, activation=tf.nn.tanh)
# act_vects = dense(dense2, 2, activation=tf.nn.tanh)

# v_dense1 = dense(obs, 32, activation=tf.nn.tanh)
# v_dense2 = dense(v_dense1, 32, activation=tf.nn.tanh)
# value = dense(v_dense2, 1)

# network = PPOTrainer(obs, act_vects, value, act_type='c', ppo_iters=20)

In [17]:
n_episodes = 1000000
max_steps = 300
update_freq = 35 # In episodes
print_freq = 5

mb = MemoryBuffer(update_freq)

In [18]:
def spawn_agent(agent_id, create_env, gen_act, mb, n_episodes=4, max_steps=500, render=False):
    env = create_env()
    for episode in range(n_episodes):
        obs = env.reset()
        mb.start_rollout(agent_id)
        for step in range(max_steps):
            act = gen_act([obs])
            
            if render:
                env.render()
                time.sleep(0.02)
                
            obs_next, rew, d, _ = env.step(act)
            
            mb.record(agent_id, obs, act, rew)
            obs = obs_next
            
            if d:
                break

In [19]:
def gather_data(mpmb, n_threads=7, n_episodes=4, return_rewards=False, reset_mem=True):
    agent_pool = []
    
    for i in range(n_threads):
        agent_pool.append(Thread(target=spawn_agent, args=(i, make_lunar_lander_c, network.gen_act, mpmb),
                                                     kwargs={'n_episodes': math.ceil(n_episodes/n_threads)}))
        agent_pool[-1].start()

    for agent in agent_pool:
        agent.join()
        
    
    if return_rewards:
        return mpmb.get_avg_reward(), mpmb.to_data(reset=reset_mem)
    return mpmb.to_data(reset=reset_mem)

In [20]:
# mpmb = MTMemoryBuffer()
# ep_rewards = []

# for i in range(n_episodes//update_freq):
# #     ep_reward, train_data = gather_data(mpmb, n_episodes=update_freq, return_rewards=True)
# #     ep_rewards.append(ep_reward)
#     spawn_agent(0, make_cart_pole, network.gen_act, mpmb, n_episodes=update_freq, max_steps=300, render=False)
#     print(mpmb.get_avg_reward())
#     train_data = mpmb.to_data()
#     network.train(train_data)
# #     if i % print_freq == 0 and i != 0:
# #         print(f'Update #{int(i)}, Recent Reward:', np.mean(ep_rewards[print_freq:]))

In [21]:
import time
total_steps = 0
total_episodes = 0
all_rewards = []

for episode in range(n_episodes):
    obs = env.reset()
    
    episode_reward = 0
    mb.start_rollout()
    for step in range(max_steps):
        act = network.gen_act([obs])
#         env.render()
#         time.sleep(0.02)
        
        obs_next, rew, d, _ = env.step(act)
        episode_reward += rew
        
        mb.record(obs, act, rew)
        obs = obs_next
        
        total_steps += 1
        if d:
            break
            
    all_rewards.append(episode_reward)
    total_episodes += 1
            
    if total_episodes % update_freq == 0:
        train_data = mb.to_data()
        network.train(train_data)
        
        if total_episodes % (update_freq * print_freq) == 0:
            print(f'Update #{int(total_episodes/update_freq)}, Recent Reward:', np.mean(all_rewards[-update_freq * print_freq:]))

Update #5, Recent Reward: 16.965714285714284
Update #10, Recent Reward: 14.714285714285714
Update #15, Recent Reward: 21.37142857142857
Update #20, Recent Reward: 21.017142857142858
Update #25, Recent Reward: 21.577142857142857
Update #30, Recent Reward: 26.205714285714286
Update #35, Recent Reward: 71.33714285714285
Update #40, Recent Reward: 20.754285714285714
Update #45, Recent Reward: 13.057142857142857
Update #50, Recent Reward: 9.462857142857143
Update #55, Recent Reward: 9.48
Update #60, Recent Reward: 9.577142857142857
Update #65, Recent Reward: 9.525714285714285
Update #70, Recent Reward: 9.48
Update #75, Recent Reward: 9.417142857142856
Update #80, Recent Reward: 9.571428571428571
Update #85, Recent Reward: 9.411428571428571
Update #90, Recent Reward: 9.525714285714285
Update #95, Recent Reward: 9.502857142857144
Update #100, Recent Reward: 9.548571428571428
Update #105, Recent Reward: 9.64
Update #110, Recent Reward: 16.777142857142856
Update #115, Recent Reward: 220.04
Upda

KeyboardInterrupt: 